In [1]:
import numpy as np
import itertools
import json

from sklearn.isotonic import IsotonicRegression
from sklearn.calibration import calibration_curve, _SigmoidCalibration, _sigmoid_calibration
from ampligraph.evaluation import evaluate_performance, mr_score, mrr_score, hits_at_n_score, generate_corruptions_for_eval
from sklearn.metrics import brier_score_loss, log_loss, accuracy_score
from scipy.special import expit

from ampligraph.datasets import load_wn11
from ampligraph.latent_features.models import TransE, ComplEx, DistMult, ConvKB, HolE
import types

In [2]:
from generate_corruptions import generate_corruptions, calibration_loss, pos_iso

In [3]:
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


In [4]:
X = load_wn11()

In [5]:
X_valid_pos = X['valid'][X['valid_labels']]
X_valid_neg = X['valid'][~X['valid_labels']]

X_test_pos = X['test'][X['test_labels']]
X_test_neg = X['test'][~X['test_labels']]

In [ ]:
losses =  ['self_adversarial', 'pairwise', 'nll', 'multiclass_nll']
models = [TransE, DistMult, ComplEx, HolE]

results = []

for m, l in itertools.product(models, losses):
    model = m(batches_count=64, seed=0, epochs=1000, k=100, eta=20,
                   optimizer='adam', optimizer_params={'lr':0.0001},
                   loss=l, verbose=False)

    model.fit(X['train'])
    
    scores = model.predict(X['test'])

    model.calibrate(X_valid_pos, batches_count=10, epochs=1000, positive_base_rate=0.5)
    print("pos", model.calibration_parameters)
    probas1 = model.predict_proba(X['test'])

    model.calibrate(X_valid_pos, X_valid_neg)
    print("pos neg", model.calibration_parameters)
    probas2 = model.predict_proba(X['test'])
    
    val_scores = model.predict(X['valid'])
    ir = IsotonicRegression(out_of_bounds='clip')
    ir.fit(np.squeeze(val_scores).astype(float), (X['valid_labels']).astype(float))
    probas3 = ir.predict(np.squeeze(scores).astype(float))
    
    model.generate_corruptions = types.MethodType(generate_corruptions, model)
    corruptions = model.generate_corruptions(X_valid_pos, batches_count=10, epochs=1000)
    val_pos_scores = np.squeeze(model.predict(X_valid_pos))
    iso_pos = pos_iso(IsotonicRegression(out_of_bounds='clip'), val_pos_scores, corruptions, positive_base_rate=0.5)
    probas4 = iso_pos.predict(np.squeeze(scores).astype(float))

    sc_pos = pos_iso(_SigmoidCalibration(), val_pos_scores, corruptions, positive_base_rate=0.5)
    print("pos sc", sc_pos.a_, sc_pos.b_)
    probas5 = sc_pos.predict(np.squeeze(scores).astype(float))
    
    val_neg_scores = np.squeeze(model.predict(X_valid_neg))
    sc_pos_neg = pos_iso(_SigmoidCalibration(), val_pos_scores, val_neg_scores, positive_base_rate=0.5)
    print("pos neg sc", sc_pos_neg.a_, sc_pos_neg.b_)
    probas6 = sc_pos_neg.predict(np.squeeze(scores).astype(float))
    
    thresholds = {r: np.median(np.sort(val_scores[X['valid'][:, 1] == r])) for r in np.unique(X['valid'][:, 1])}
    thresholds_test = np.vectorize(thresholds.get)(X['test'][:, 1])
    per_relation_acc = accuracy_score(X['test_labels'], scores > thresholds_test)

    acc_uncalib = accuracy_score(X['test_labels'], expit(scores) > 0.5)
    
    acc1 = accuracy_score(X['test_labels'], probas1 > 0.5)
    acc2 = accuracy_score(X['test_labels'], probas2 > 0.5)
    acc3 = accuracy_score(X['test_labels'], probas3 > 0.5)
    acc4 = accuracy_score(X['test_labels'], probas4 > 0.5)
    acc5 = accuracy_score(X['test_labels'], probas5 > 0.5)
    acc6 = accuracy_score(X['test_labels'], probas6 > 0.5)
    
    filter_triples = np.concatenate((X['train'], X_valid_pos, X_test_pos))
    ranks = evaluate_performance(X_test_pos, 
                                 model=model, 
                                 filter_triples=filter_triples,
                                 use_default_protocol=True, 
                                 verbose=False)

    results.append({
        'model': m.__name__,
        'loss': l,
        'brier_score_scores': brier_score_loss(X['test_labels'], expit(scores)),
        'log_loss_scores': log_loss(X['test_labels'], expit(scores), eps=1e-7),
        'brier_score_probas_pos': brier_score_loss(X['test_labels'], probas1),
        'log_loss_probas_pos': log_loss(X['test_labels'], probas1, eps=1e-7),
        'brier_score_probas_pos_neg': brier_score_loss(X['test_labels'], probas2),
        'log_loss_probas_pos_neg': log_loss(X['test_labels'], probas2, eps=1e-7),
        'brier_score_probas_pos_neg_iso': brier_score_loss(X['test_labels'], probas3),
        'log_loss_probas_pos_neg_iso': log_loss(X['test_labels'], probas3, eps=1e-7),
        'brier_score_probas_pos_iso': brier_score_loss(X['test_labels'], probas4),
        'log_loss_probas_pos_iso': log_loss(X['test_labels'], probas4, eps=1e-7),
        'brier_score_probas_pos_sc': brier_score_loss(X['test_labels'], probas5),
        'log_loss_probas_pos_sc': log_loss(X['test_labels'], probas5, eps=1e-7),
        'brier_score_probas_pos_neg_sc': brier_score_loss(X['test_labels'], probas6),
        'log_loss_probas_pos_neg_sc': log_loss(X['test_labels'], probas6, eps=1e-7),
        'ece_scores': calibration_loss(X['test_labels'], expit(scores)),
        'ece_probas_pos': calibration_loss(X['test_labels'], probas1),
        'ece_probas_pos_neg': calibration_loss(X['test_labels'], probas2),
        'ece_probas_pos_neg_iso': calibration_loss(X['test_labels'], probas3),
        'ece_probas_pos_iso': calibration_loss(X['test_labels'], probas4),
        'ece_probas_pos_sc': calibration_loss(X['test_labels'], probas5),
        'ece_probas_pos_neg_sc': calibration_loss(X['test_labels'], probas6),
        'metrics_mrr': mrr_score(ranks), 
        'metrics_hits@10': hits_at_n_score(ranks, n=10),
        'metrics_mr': mr_score(ranks),
        'accuracy_per_relation': per_relation_acc,
        'accuracy_uncalib': acc_uncalib,
        'accuracy_pos': acc1,
        'accuracy_pos_neg': acc2,
        'accuracy_pos_neg_iso': acc3,
        'accuracy_pos_iso': acc4,
        'accuracy_pos_sc': acc5,
        'accuracy_pos_neg_sc': acc6
    })
        
    print(json.dumps(results[-1], indent=2))

In [7]:
import pandas as pd

In [8]:
def highlight_min(s):
    is_min = s == s.min()
    return ['font-weight: bold' if v else '' for v in is_min]

In [17]:
df = pd.DataFrame(results).set_index(['model', 'loss'])
df.to_csv("main_results_wn11.csv", index=False)

In [10]:
bs = df[(c for c in df.columns if c.startswith('brier'))]
bs.columns = [c[len("brier_score_"):] for c in bs.columns]
bs.style.apply(highlight_min, axis=1)

In [11]:
ll = df[(c for c in df.columns if c.startswith('log_loss'))]
ll.columns = [c[len("log_loss_"):] for c in ll.columns]
ll.style.apply(highlight_min, axis=1)

In [12]:
print((bs.reset_index()
 .query("loss == 'self_adversarial' ")
 [['model', 'scores', 'probas_pos_neg', 'probas_pos_neg_iso', 'probas_pos', 'probas_pos_iso']]
 .reset_index(drop=True)
 .round(3)
 .to_latex()))

\begin{tabular}{llrrrrr}
\toprule
{} &     model &  scores &  probas\_pos\_neg &  probas\_pos\_neg\_iso &  probas\_pos &  probas\_pos\_iso \\
\midrule
0 &    TransE &   0.443 &           0.089 &               0.087 &       0.091 &           0.087 \\
1 &  DistMult &   0.488 &           0.213 &               0.208 &       0.214 &           0.208 \\
2 &   ComplEx &   0.490 &           0.240 &               0.228 &       0.240 &           0.228 \\
3 &      HolE &   0.474 &           0.235 &               0.235 &       0.235 &           0.236 \\
\bottomrule
\end{tabular}



In [13]:
print(ll.reset_index()
 .query("loss == 'self_adversarial' ")
 [['model', 'scores', 'probas_pos_neg', 'probas_pos_neg_iso', 'probas_pos', 'probas_pos_iso']]
 .reset_index(drop=True)
  .round(3)
 .to_latex())

\begin{tabular}{llrrrrr}
\toprule
{} &     model &  scores &  probas\_pos\_neg &  probas\_pos\_neg\_iso &  probas\_pos &  probas\_pos\_iso \\
\midrule
0 &    TransE &   1.959 &           0.302 &               0.295 &       0.308 &           0.296 \\
1 &  DistMult &   5.625 &           0.618 &               0.604 &       0.618 &           0.601 \\
2 &   ComplEx &   6.061 &           0.674 &               0.651 &       0.674 &           0.650 \\
3 &      HolE &   2.731 &           0.663 &               0.661 &       0.663 &           0.668 \\
\bottomrule
\end{tabular}



In [14]:
print(bs.reset_index()
 .query("model == 'TransE' ")
 [['loss',  'probas_pos_neg', 'probas_pos_neg_iso', 'probas_pos', 'probas_pos_iso']]
 .reset_index(drop=True)
  .round(3)
 .to_latex())

\begin{tabular}{llrrrr}
\toprule
{} &              loss &  probas\_pos\_neg &  probas\_pos\_neg\_iso &  probas\_pos &  probas\_pos\_iso \\
\midrule
0 &  self\_adversarial &           0.089 &               0.087 &       0.091 &           0.087 \\
1 &          pairwise &           0.202 &               0.198 &       0.209 &           0.200 \\
2 &               nll &           0.093 &               0.088 &       0.093 &           0.088 \\
3 &    multiclass\_nll &           0.204 &               0.188 &       0.204 &           0.189 \\
\bottomrule
\end{tabular}



In [15]:
print(ll.reset_index()
 .query("model == 'TransE' ")
 [['loss', 'probas_pos_neg', 'probas_pos_neg_iso', 'probas_pos', 'probas_pos_iso']]
 .reset_index(drop=True)
  .round(3)
 .to_latex())

\begin{tabular}{llrrrr}
\toprule
{} &              loss &  probas\_pos\_neg &  probas\_pos\_neg\_iso &  probas\_pos &  probas\_pos\_iso \\
\midrule
0 &  self\_adversarial &           0.302 &               0.295 &       0.308 &           0.296 \\
1 &          pairwise &           0.591 &               0.585 &       0.606 &           0.589 \\
2 &               nll &           0.342 &               0.299 &       0.342 &           0.299 \\
3 &    multiclass\_nll &           0.599 &               0.549 &       0.599 &           0.550 \\
\bottomrule
\end{tabular}



In [25]:
acc = df[(c for c in df.columns if c.startswith('accuracy'))]
acc.columns = [c[len("accuracy_"):] for c in acc.columns]
acc.style.apply(highlight_min, axis=1)

In [26]:
print((acc*100).reset_index()
 .query("loss == 'self_adversarial' ")
 [['model', 'pos_neg', 'pos_neg_iso', 'pos', 'pos_iso', 'uncalib', 'per_relation']]
 .reset_index(drop=True)
  .round(1)
 .to_latex())

\begin{tabular}{llrrrrrr}
\toprule
{} &     model &  pos\_neg &  pos\_neg\_iso &   pos &  pos\_iso &  uncalib &  per\_relation \\
\midrule
0 &    TransE &     88.8 &         88.9 &  88.9 &     88.9 &     50.7 &          88.2 \\
1 &  DistMult &     66.5 &         67.2 &  66.4 &     67.1 &     50.8 &          67.2 \\
2 &   ComplEx &     60.6 &         62.4 &  60.0 &     62.4 &     50.8 &          59.6 \\
3 &      HolE &     59.3 &         59.0 &  59.3 &     59.0 &     50.9 &          60.8 \\
\bottomrule
\end{tabular}



In [21]:
metrics = df[(c for c in df.columns if c.startswith('metrics'))]
metrics.columns = [c[len("metrics_"):] for c in metrics.columns]
metrics

hits@10            mr       mrr
model    loss                                              
TransE   self_adversarial  0.308572   2288.616938  0.155053
         pairwise          0.134247   7259.422007  0.057914
         nll               0.226973   2514.538224  0.133938
         multiclass_nll    0.234134   7455.075314  0.108405
DistMult self_adversarial  0.081290   9999.898825  0.044532
         pairwise          0.092881  12092.537606  0.050656
         nll               0.092108  12594.588038  0.049014
         multiclass_nll    0.108902  11518.330105  0.060369
ComplEx  self_adversarial  0.093859  13814.839738  0.053883
         pairwise          0.114053  11376.982124  0.067699
         nll               0.108335  13737.130126  0.063321
         multiclass_nll    0.141407  10967.958531  0.085778
HolE     self_adversarial  0.035236  13354.698949  0.017294
         pairwise          0.172265  10329.089481  0.112900
         nll               0.115290  13999.438389  0.068480
         multiclass_nll    0.189264   9799.101432  0.123030

In [22]:
print(metrics.reset_index()
 .query("loss == 'self_adversarial' ")
 [['model', 'mr', 'mrr', 'hits@10']]
 .reset_index(drop=True)
  .round(3)
 .to_latex())

\begin{tabular}{llrrr}
\toprule
{} &     model &         mr &    mrr &  hits@10 \\
\midrule
0 &    TransE &   2288.617 &  0.155 &    0.309 \\
1 &  DistMult &   9999.899 &  0.045 &    0.081 \\
2 &   ComplEx &  13814.840 &  0.054 &    0.094 \\
3 &      HolE &  13354.699 &  0.017 &    0.035 \\
\bottomrule
\end{tabular}



In [ ]:
def highlight_max(s):
    is_min = s == s.max()
    return ['font-weight: bold' if v else '' for v in is_min]

acc = df[(c for c in df.columns if c.startswith('accuracy'))]
acc.columns = [c[len("accuracy_"):] for c in acc.columns]
acc.style.apply(highlight_max, axis=1)

In [ ]:
df.corr(method='spearman').reset_index().query("index.str.startswith('accuracy')")[['index', 'log_loss_probas_pos_neg', 'log_loss_probas_pos_neg_iso', 'log_loss_probas_pos', 'log_loss_probas_pos_iso']]

In [ ]:
df.corr(method='spearman').reset_index().query("index.str.startswith('accuracy')")[['index', 'brier_score_probas_pos_neg', 'brier_score_probas_pos_neg_iso', 'brier_score_probas_pos', 'brier_score_probas_pos_iso']]

In [18]:
model = TransE(batches_count=64, seed=0, epochs=1000, k=100, eta=20,
               optimizer='adam', optimizer_params={'lr':0.0001},
               loss='self_adversarial', verbose=False)

model.fit(X['train'])

scores = model.predict(X['test'])

model.calibrate(X_valid_pos, batches_count=10, epochs=1000, positive_base_rate=0.5)
print("pos", model.calibration_parameters)
probas1 = model.predict_proba(X['test'])

model.calibrate(X_valid_pos, X_valid_neg)
print("pos neg", model.calibration_parameters)
probas2 = model.predict_proba(X['test'])

val_scores = model.predict(X['valid'])

thresholds = {r: np.median(np.sort(val_scores[X['valid'][:, 1] == r])) for r in np.unique(X['valid'][:, 1])}
thresholds_test = np.vectorize(thresholds.get)(X['test'][:, 1])
per_relation_acc = accuracy_score(X['test_labels'], scores > thresholds_test)

print(thresholds)


pos [-1.1740658, -6.8939967]
pos neg [-1.4084864, -8.3745365]
{'_domain_region': -6.0069733, '_domain_topic': -5.5207396, '_has_instance': -6.2901406, '_has_part': -5.673306, '_member_holonym': -6.3117476, '_member_meronym': -5.982978, '_part_of': -5.798244, '_similar_to': -6.852225, '_subordinate_instance_of': -5.4750223, '_synset_domain_topic': -6.6392403, '_type_of': -6.743014}


In [20]:
per_relation_acc

0.8819648837917385